In [1]:
import cantera as ct
import numpy as np

In [2]:
gas=ct.Solution("gri30.cti")

In [3]:
ics=[13, 1, 3, 2, 47, 8]
ns=len(gas.species())
species=gas.species_names
multiindex=np.zeros(ns)
for i in range(0,len(ics),2):
    multiindex[int(ics[i])]=ics[i+1]
gas.TPX=1500,1*ct.one_atm,multiindex

In [4]:
rind=32
reaction=gas.reactions()[32]
reaction

<ThreeBodyReaction: H + O2 + M <=> HO2 + M>

In [5]:
gas.forward_rate_constants[rind]

3838115.667757274

In [6]:
k0=reaction.rate.pre_exponential_factor*gas.T**(reaction.rate.temperature_exponent)*np.exp(-reaction.rate.activation_energy/(ct.gas_constant*gas.T))

In [7]:
k0/gas.forward_rate_constants[rind]

1353.9464559585504

In [8]:
stoi=np.zeros(ns)
k=0
for third_body in range(ns):
    efficiency=reaction.default_efficiency
    stoi[third_body]+=1
    if species[third_body] in reaction.efficiencies.keys():
        efficiency=reaction.efficiencies[species[third_body]]
    #since the k are dimensionalized, we are missing a dimension here
    #concentrations should be moles per unit volume
    #we only know the mole fractions
    #is there some presumed volume for the gas??
    #well, PV=nRT..., so RT/P is natural volume per moles, indeed
    k+=efficiency*k0*np.product(gas.X**(stoi))/(ct.gas_constant*gas.T/gas.P)
    stoi[third_body]-=1

In [9]:
print(k/gas.forward_rate_constants[rind])

1.0000000000000009


In [10]:
gas.volume_mole

123.08604145077719

In [11]:
ct.gas_constant*gas.T/gas.P

123.08604145077719

In [31]:
gas.TPX=1500,1*ct.one_atm,multiindex

refquant=ct.Quantity(gas,moles=1)
refenth=refquant.enthalpy
refenergy=refquant.int_energy
refmass=refquant.mass
refvol=refquant.volume

gas2=ct.Solution("gri30.cti")
gas2.UVX=refenergy/refmass,refvol/refmass,multiindex
quant=ct.Quantity(gas2,moles=1)

print(quant.P/ct.one_atm, quant.T)
print(refquant.P/ct.one_atm, refquant.T)

1.0000000000000004 1500.0
1.0000000000000002 1500.0


In [26]:
multiindex

array([0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 8., 0., 0., 0.,
       0., 0.])

In [38]:
gas=ct.Solution('h2o2.cti')
gas.TPX=268.431,0.178954*ct.one_atm,np.array([4,2,1,1,2,1,0,1,3])
print(np.max(gas.forward_rate_constants),np.max(gas.reverse_rate_constants))
gas.TPX=7.60689,0.0050712*ct.one_atm,np.array([4,2,1,1,2,1,0,1,3])
print(np.max(gas.forward_rate_constants),np.max(gas.reverse_rate_constants))

160555638441.48444 21894200529.856678
1.9397024486818256e+64 4.0900158173183055e-125


In [41]:
gas.reactions()[np.argmax(gas.forward_rate_constants)]

<ElementaryReaction: 2 OH <=> H2O + O>

In [50]:
gas=ct.Solution('gri30.cti')
gas.species_names[47]

'N2'